# Combining *PyDIP* and *NumPy*
The following are some examples of using the *DIPlib* Python binding, *PyDIP*.

In [1]:
import diplib as dip
import numpy as np

DIPlib -- a quantitative image analysis library
Version 3.4.0 (Nov 30 2022)
For more information see https://diplib.org


## Converting between images and arrays
Casting a *NumPy* array to a *PyDIP* image does not incur a data copy. The array and the image will share data.

In [2]:
arr = np.zeros((3, 4))
img = dip.Image(arr)
img[0, 0] = 1
print(arr[0, 0])

1.0


For the reverse operation, the same is true when using `np.asarray()`, or `np.array(..., copy=False)`.

In [3]:
img = dip.Image((5, 6), 1, "UINT8")
arr = np.asarray(img)
arr[2, 3] = 10
print(img[3, 2])

[10]


As you've noticed above, we reversed the indexing. *PyDIP*'s first dimension is horizontal (x), the second is vertical (y), etc. In *NumPy* the last dimensin is horizontal, the second to last vertical, etc.

This can be confusing when combining *PyDIP* functionality with other image processing packages such as *scikit-image*. For this situation, it is possible to reconfigure *PyDIP* to reverse its dimensions, to match *NumPy*'s (we won't call this function right now).

In [4]:
help(dip.ReverseDimensions)

Help on built-in function ReverseDimensions in module diplib.PyDIP_bin:

ReverseDimensions(...) method of builtins.PyCapsule instance
    ReverseDimensions() -> None
    
    By default, DIPlib uses the (x,y,z) index order. This order is reversed from
    how NumPy (and, by extension, packages such as scikit-image, matplotlib and
    imageio) index. Calling `ReverseDimensions()` causes DIPlib, for the remainder
    of the session, to also follow the (z,y,x) index order, giving dimensions the
    same order as Python users might be used to. Use this function at the top of
    your program, right after importing the `diplib` package. There is no way to
    revert to the default order. Please don't try to mix dimension ordering within
    your program.
    
    See dip.AreDimensionsReversed().



When converting a tensor image (e.g. a color image) to an array, the tensor dimension will be the last array dimension.

In [5]:
img = dip.Image((50, 60), 3, "UINT8")
arr = np.asarray(img)
print(arr.shape)

(60, 50, 3)


When converting an array to an image, there is no indication of whether there is a tensor dimension, and which one it is. So, *PyDIP* uses a simple heuristic to guess if the array represents a tensor (or color) image:
 - if the array has more than two dimensions, and
 - the first or the last dimension has 4 or fewer elements,
then that dimension is the tensor dimension. Thus, only the first or last dimension is ever considered. We chose 4 as a threshold to catch both RGB and RGBA images.

In [6]:
a = dip.Image(arr)
print(a.__repr__())

<Tensor image (3x1 column vector, 3 elements), UINT8, sizes {50, 60}>


Note that the image is not marked as being a color image. One has to manually add this informaiton if it is relevant:

In [7]:
a = dip.Image(arr)
a.SetColorSpace('sRGB')
print(a.__repr__())

<Color image (3x1 column vector, 3 elements, sRGB), UINT8, sizes {50, 60}>


## Calling *NumPy* functions with an image
When calling a *NumPy* with a `dip.Image` object as input, the output of the *NumPy* function will be a *NumPy* array, just as if the function had been called with an array as input.

In [8]:
a = np.ravel(img)
print(type(a))
print(a.shape)
print(a.dtype)

<class 'numpy.ndarray'>
(9000,)
uint8


## Calling PyDIP functions with an array
When calling a *PyDIP* function with a `np.array` object as input, the output will be a `dip.Image` object, just as if the function had been called with an array as input. In fact, the array is cast to an image exactly as described earlier.

In [9]:
a = dip.Gauss(arr)
print(a.__repr__())

<Tensor image (3x1 column vector, 3 elements), SFLOAT, sizes {50, 60}>


The `out` keyword argument to *PyDIP* functions is easier to use with a `dip.Image` object than with a `np.array` object. When passing in an image object, the image can be reforged (meaning its data segment can be reallocated, changing the size, number of tensor elements, and/or data type). The same is not true for an input of a different type, which will be converted to a protected `dip.Image` object (see [the documentation on the protect flag](https://diplib.org/diplib-docs/dip-Image.html#protect)). This means that it must have the right sizes to receive the output of the function. 

In [10]:
a = np.zeros(10)
try:
    dip.Gauss(img, out=a)
except Exception as e:
    print(f"\033[31m{type(e).__name__} exception:", str(e).split('\n')[0])

RuntimeError exception: Image is protected


In [11]:
a = np.zeros((60, 50, 3))
dip.Gauss(img, out=a)

Note how the filter is computed in the output type, 64-bit float in this case.

This is most useful to work in-place, where the feature is easy to use:

In [12]:
dip.Gauss(arr, out=arr)